In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '20'

import numpy as np
import pandas as pd
import re
import csv
from parsivar import Normalizer
import tensorflow_text as tf_text

In [12]:
def count_english_words(row):
    if row is None:
        return 10
    words = row.split(' ')
    cnt = 0
    for word in words:
        if re.match('[a-zA-z]+', word):
            cnt += 1
            
    return cnt

In [13]:
def read_data_from_dir(directory='./'):
    files = os.listdir(directory)
    data = []
    for file in files:
        if 'done' not in file: 
            continue
        with open(file, 'r') as txtfile:
            data.append(txtfile.read())

    data = '\n'.join(data).split('\n')
    data = [x.split('\t')[:2] for x in data]
    df = pd.DataFrame(data, columns=['Source', 'Target'])
    return df

In [14]:
MAX_SOURCE_LEN = 50

fa_normalizer = Normalizer()

data = read_data_from_dir()
data['Source'] = data['Source'].str.strip().replace('...', ' ')
data['Target'] = data['Target'].str.strip().replace('...', ' ')
data['Target'] = data['Target'].map(lambda x: fa_normalizer.normalize(x) if x is not None else None)
data = data[data['Target'].map(lambda x: count_english_words(x) == 0)]
data['Target'] = data['Target'].map(lambda x: re.sub('\u200c', ' ', x))
data = data[data['Source'].map(lambda x: len(x.split(' ')) <= MAX_SOURCE_LEN)]
data = data.drop_duplicates(keep='first')
print(len(data))
data = data.dropna()
data

994267


,Source,Target
0,"I am not like you, viscount; you like your title",من مثل تو نیستم ، ویسکونت . عنوان خود را دوست ...
1,do you not?,آیا شما نه ؟
2,"Certainly, replied Albert",مطمئنا ، آلبرت پاسخ داد
3,seeing that without my title I should be nothing;,می بینم که بدون عنوان من نباید هیچ باشم .
4,"while you, sacrificing the baron",در حالی که شما ، بارون را قربانی می کنید
...,...,...
1583774,while it is within our reach,در حالی که در دسترس ماست
1583775,"Well, sir, said Danglars, in case your proposals",خب ، آقا ، گفت : دنگلر ، در صورت پیشنهاد شما
1583776,which do me honor,که به من افتخار می کنند
1583777,are accepted by my wife and daughter,توسط همسر و دخترم پذیرفته شده اند


In [15]:
data.to_csv('combined.txt', sep='\t', header=False, index=False, encoding='utf-8')